In [ ]:
# !pip install shapely
# !pip install plotly
!pip install geopandas

In [1]:
from shapely.geometry import shape
import plotly.express as px
import geopandas as gpd
import json
from shapely.geometry import Polygon, Point
from shapely import wkt
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os

In [15]:
# geojson_path = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/Visium_HD_Human_Colon_Cancer-wsi-40598_0_65263_22706-landmarks.geojson"
geojson_path = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/Visium_HD_Mouse_Small_Intestine-wsi-156_4_23459_24009_all_for_one.geojson"

segmentation_df_path = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/predictions/stardist_cells_df.csv"
results_eval_dir = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/mouse_anatomical_landmark_eval"
os.makedirs(results_eval_dir, exist_ok=True)
# predictions_df_path = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/predictions/Sargent+naive.csv"
# predictions_df_path = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/predictions/Sargent+weighted.csv"
# predictions_df_path = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/predictions/cellassign+weighted.csv"
# predictions_df_path = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/predictions/cellassign+naive.csv"
# predictions_df_path = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/predictions/Sargent+weighted-full.csv"
predictions_df_path = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/predictions/sargent+weighted+mouse.csv"


method = predictions_df_path.split("/")[-1].split(".")[0]

segmentation_df = pd.read_csv(segmentation_df_path)
predictions_df = pd.read_csv(predictions_df_path)
predictions_df = predictions_df.merge(segmentation_df[["id", "geometry"]], how="left", on="id")
predictions_df = predictions_df[~predictions_df.geometry.isna()]
try:
    predictions_df["geometry"] = predictions_df["geometry"].apply(wkt.loads)
except:
    pass
pred_gpd = gpd.GeoDataFrame(predictions_df,geometry="geometry")

In [16]:
annotation_names = []
annotation_geometries = []
with open(geojson_path) as f:
    regions = json.load(f)
for region in regions["features"]:
    ann_type = region["properties"]["objectType"]
    if ann_type == "annotation":
        annotation_name = region["properties"]["classification"]["name"]
        if annotation_name in ["Region*"]:
            continue
        annotation_geometries.append(shape(region["geometry"]))
        annotation_names.append(annotation_name)
annotations_gpd = gpd.GeoDataFrame({"geometry": annotation_geometries, "label": annotation_names})
annotations_gpd["ann_ix"] = [f"ID_{i}" for i in range(len(annotations_gpd))]
cells_within_ann_gpd = gpd.sjoin(pred_gpd[["cell_type", "cell_x", "cell_y", "geometry", "id"]], annotations_gpd, how='left', predicate='within')
cells_within_ann_gpd = cells_within_ann_gpd.drop_duplicates(subset=["id"])

In [ ]:
for annotation_name in annotation_names:
    df = cells_within_ann_gpd[cells_within_ann_gpd.label == annotation_name]
    # df = df[~(df.cell_type == "unclassified")]
    df = df.groupby(["cell_type"]).agg("count").reset_index()
    df = df.sort_values("id", ascending=False)
    fig = px.bar(df, x='cell_type', y='id', title=f"Region: {annotation_name}")
    fig.update_layout(
        xaxis_title="cell type", yaxis_title="# cells"
    )
    fig.show()
    fig.write_html(os.path.join(results_eval_dir, f"{method}_{annotation_name}_cell_counts.html"))

In [ ]:
results_eval_dir